In [1]:
from typing import Tuple

import numpy as np
import sklearn, sklearn.model_selection, sklearn.datasets
import matplotlib.pyplot as plt

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier

In [2]:
data = sklearn.datasets.fetch_california_housing(as_frame=True).frame
x_keys = ['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup', 'Latitude', 'Longitude']
y_key = 'MedHouseVal'
data[y_key] = data[y_key] > data[y_key].mean()

In [3]:
np.random.seed(0) # for reproducibility

a) Split the data set into training (80%) and test set (20%).  

Implement the provided template function for fitting a model on the train set and evaluating it on the train and test sets.

In [4]:
def fit_and_evaluate_model(model) -> Tuple[float, float]:
    """Fit and evaluate a model on the training and test sets.

    Returns:
        Tuple[float, float]: The training and test error.
    """
    # Fit the model on the training set
    

    # Predict the labels on the training and test sets

    # Compute the training and test error
    train_error = None
    test_error = None
    return train_error, test_error

b) Train a decision tree classifier with a maximum depth of 10 on the training set. Compute the error on the train set and the error on the test set.

c) Visualize the tree. How many nodes can it have at maximum? How many nodes does your tree have?

*The number of nodes can grow up to $\sum_{i = 1}^{10} 2^{i-1} = 2^{10} - 1 = 1023$.*

c) Train multiple trees with depths between 1 and 30 and visualize the training and test errors.
What do you observe?

d) Train a bagging classifier with 10 trees of maximum depth 10. Compute the train and test errors.

Train multiple bagging classifiers with 1 to 30 base estimators, each of depth 10. Visualize the curves for training and test errors.

e) Train a boosting classifier (`sklearn.ensemble.AdaBoostClassifier`) with 10 trees of maximum depth 10. Compute the train and test errors.

Train multiple boosting classifiers with 1 to 30 base estimators, each of depth 5. Visualize the curves for training and test errors.

f) Train a random forest classifier with 10 trees and max_depth=10. Compute the train and test errors.

Train multiple random forests with 1 to 30 base estimators, each of depth 5. Visualize the curves for training and test errors.